<a href="https://colab.research.google.com/github/huongd17at089/rice_diseases_detection/blob/huong/rice_diseases_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!rm -rf /content/sample_data
! pip install -q kaggle

In [ ]:
# from google.colab import files
# files.upload()
# {"username":"dinhgihuong","key":"a9a05d71e7c7e2449d5d56f66ba6c2aa"}  kaggle.json

In [2]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
# ! kaggle datasets list

In [3]:
!kaggle datasets list -s "Rice Diseases"

ref                                                             title                                            size  lastUpdated          downloadCount  
--------------------------------------------------------------  ----------------------------------------------  -----  -------------------  -------------  
minhhuy2810/rice-diseases-image-dataset                         Rice Diseases Image Dataset                      12GB  2019-11-15 22:17:33           7921  
vbookshelf/rice-leaf-diseases                                   Rice Leaf Diseases Dataset                       37MB  2020-02-21 01:35:02           2030  
shayanriyaz/riceleafs                                           Rice Leafs                                       15GB  2019-07-18 11:31:17           1811  
vipoooool/new-plant-diseases-dataset                            New Plant Diseases Dataset                        3GB  2018-11-18 07:09:16          13998  
pitt/contagious-diseases                                        

In [4]:
! kaggle datasets download -d "minhhuy2810/rice-diseases-image-dataset"

100% 12.0G/12.0G [04:54<00:00, 6.28MB/s]
100% 12.0G/12.0G [04:54<00:00, 43.7MB/s]


In [ ]:
! mkdir data
! unzip rice-diseases-image-dataset.zip -d data
!rm -rf /content/data/LabelledRice
!rm rice-diseases-image-dataset.zip

In [25]:
import os
import numpy as np
import cv2

data_path = "/content/data/RiceDiseaseDataset/"
img_size = 512
label_dic = {
    "Healthy" : 0,
    "BrownSpot": 1,
    "Hispa": 2,
    "LeafBlast" : 3

}
n_class = len(label_dic.keys)

In [14]:
def get_data(path, img_size):
  data = []
  for root, dirs, files in os.walk(path):
    label = label_dic.get(list(root.split(sep="/"))[-1], None)
    if label :
      for f in files:
        try: 
          img_path = os.path.join(root,f)
          img = cv2.imread(img_path) #cv2.IMREAD_GRAYSCALE
          img = cv2.resize(img,(img_size,img_size))
          data.append([img, label])
        except Exception as e:
          print(e)
  return np.array(data)

In [ ]:
def data_preprocessing(data):
  imgs = data[:,0]
  labels = data[:,1]
  

In [26]:
data_train = get_data(data_path + "train/", img_size)
data_test = get_data(data_path + "validation/", img_size)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Dropout, Flatten

def createModel():
    model = Sequential()
    model.add(Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=(img_size,img_size,3)))
    model.add(Conv2D(32, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(n_class, activation='softmax'))
    
    return model